In [ ]:
import camelot 
import pandas as pd
import numpy as np
import matplotlib
#Pra resolver o problema de UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown plt.show())
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt


### Extração de dados

In [ ]:
pdf_file = 'extrato.pdf'

In [ ]:
tables = camelot.read_pdf(pdf_file, pages='all', flavor='stream', table_regions=['44,688,552,145'], columns=['96,289,379,444,513'])


Pra poder pegar as coordenadas das colunas no PDF:

In [ ]:
#camelot.plot(tables[3], kind='grid').show()

In [ ]:
tables

In [ ]:
tables[0].df


### Transformação

#### Juntar todas as tabelas em uma só

In [ ]:
df_list = []

In [ ]:
# Iterar sobre as tabelas e converter cada uma para DataFrame
for table in tables:
    df = table.df  # A tabela já é um DataFrame
    df_list.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df = pd.concat(df_list, ignore_index=True)

#### Resetar o cabeçalho do Dataframe

In [ ]:
df.columns = df.iloc[0]
df = df.drop(0).reset_index(drop=True)


In [ ]:
df.head()

#### Dropar linhas de cabeçalho das outras páginas

In [ ]:
palavras = 'Data|Histórico|Docto\.|Crédito \(R\$\)|Débito \(R\$\)|Saldo \(R\$\)'

# Filtrando as linhas que não contêm essas palavras
df = df[~df.apply(lambda row: row.astype(str).str.contains(palavras, case=False).any(), axis=1)]


#df.to_excel('dataframe_teste.xlsx', index=False)

#Fazendo a pesquisa de uma palavra específica 

count = df['Data'].str.contains('Data', case=False, na=False).sum()

print(count)

#### Concatenar linhas de 3 em 3 linhas

Na hora da extração um único dado é repartido em 3 linhas, então tenho que concatenar essas linhas

In [ ]:
# Lista para armazenar as linhas combinadas
resultados = []

# Iterando sobre o DataFrame de 3 em 3
for i in range(0, len(df), 3):
    # Selecionando o grupo de 3 linhas
    grupo = df.iloc[i:i+3]
    
    # Concatenando os valores das colunas, ignorando valores vazios (NaN ou None)
    resultado = grupo.apply(lambda col: ' '.join(col.dropna()), axis=0)
    
    
    # Adicionando o resultado à lista
    resultados.append(resultado)

# Criando um novo DataFrame com os resultados
df_juntado = pd.DataFrame(resultados)

df_juntado.head()

#### Remover espaços e simbolos antes das palavras


In [ ]:
df_juntado.info()

In [ ]:
import re

# Função para remover espaços e símbolos antes das palavras
def limpar_espacos_simbolos(valor):
    if isinstance(valor, str):  # Verifica se o valor é uma string
        # Remove qualquer caractere que não seja letra ou número antes ou depois da palavra
        valor = re.sub(r'^[^\w]+', '', valor)  # Remove qualquer símbolo ou espaço no início
        valor = re.sub(r'[^\w]+$', '', valor)  # Remove qualquer símbolo ou espaço no final
        return valor
    return valor  # Caso não seja string, retorna o valor original


# Aplicando a função em todo o DataFrame
df_limpo = df_juntado.map(limpar_espacos_simbolos)

# Exibindo o DataFrame limpo
df_limpo

#### Preencher espaços de data vazios

In [ ]:
# Certifique-se de que a coluna 'Data' está no tipo datetime
df_limpo['Data'] = pd.to_datetime(df_limpo['Data'], errors='coerce')

last_data = None  # Variável para armazenar a última data encontrada

# Iterar pelas linhas do DataFrame
for index, row in df_limpo.iterrows():
    if pd.notna(row['Data']) and row['Data'] is not None:  # Verifica se a célula da coluna "Data" não for vazia (NaN ou None)
        last_data = row['Data']  # Atualiza a última data
    elif (pd.isna(row['Data']) or row['Data'] is None) and last_data is not None:  # Se a célula de "Data" for vazia (NaN ou None)
        df_limpo.at[index, 'Data'] = last_data  # Preenche a célula vazia com a última data

# Formatar as datas para 'dd/mm/yyyy'
df_limpo['Data'] = df_limpo['Data'].dt.strftime('%d/%m/%Y')

# Garantir que a última linha tenha 'Total' na coluna 'Data'
df_limpo.at[df_limpo.index[-1], 'Data'] = 'Total'

df_limpo.head()

In [ ]:
def converter_para_float(valor):
    if isinstance(valor, str):
        if valor == '':  # Verifica se é uma string vazia
            return np.nan  # Retorna NaN para string vazia
        # Substitui a vírgula por ponto e remove os pontos de milhar
        valor = valor.replace('.', '').replace(',', '.')
        return float(valor)
    return valor


df_limpo['Crédito (R$)'] = df_limpo['Crédito (R$)'].apply(converter_para_float)
df_limpo['Débito (R$)'] = df_limpo['Débito (R$)'].apply(converter_para_float)
df_limpo['Saldo (R$)'] = df_limpo['Saldo (R$)'].apply(converter_para_float)

In [ ]:
df_limpo.info()

### Carregamento

#### Carregar dados em um csv

In [ ]:
# df_limpo.to_csv('dataframe_completo.csv', index=False)

#### Carregar dados em um excel


In [ ]:

df_limpo.to_excel('dataframe_completo.xlsx', index=False)